# 5. 数据类构建器

> Python 提供了几种构建**简单类**的方法, 这些类只是 <mark>字段</mark> 的容器，几乎没有额外功能。这种模式称为「数据类」(data class)

以下三种<mark>类构建器</mark>可简化「数据构建类」的构建过程
- `collections.namedtuple`
- `typing.NamedTuple`
- `@dataclasses.dataclass`
    - 与前两种方式相比，可定制的内容更多，增加了大量选项，可以实现更复杂的功能，从 Python3.7 开始提供

> 本章介绍完这些类构建器之后，会讨论为什么数据类模式是一种代码异味，它的出现可能意味着面向对象设计欠缺

## 5.2 数据类构建器的概述

一个表示地理位置经纬度的简单类

In [1]:
class Coordinate:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

- `Coordinate` 类的作用是保存经纬度属性
- 为 `__init__` 方法编写样板代码容易乏味，尤其属性多的时候（每一个属性都要写3次【函数签名、self 中两次】）
- 更糟的是，样板代码没有给我们提供 Python 对象都有的基本功能

In [3]:
moscow = Coordinate(55.751244, 37.618423)
print(moscow) # *1

location = Coordinate(55.751244, 37.618423)

print(moscow == location) # *2
print((location.lat, location.lon) == (moscow.lat, moscow.lon)) # *3

False
True


上面代码中注释标记处

*1 继承 object 的 `__repr__` 的作用不大

*2 `==` 没有意义，因为继承自 `object` 的 `__eq__` 方法比较对象的 ID

*3 相比较两个地理位置的经纬度，只能一一比较各个属性

> 数据类构建器自动提供必要的 `__init__`, `__repr__`, `__eq__`等方法，还有别的有用的功能

<span style="color:green"> 本章的类构建器都不依赖继承。

- `collections.namedtuple` 和 `typing.NamedTuple`构建的类都是 tuple 的子类
- `@dataclass` 是类装饰器，不影响类层次结构
- 这三个类构建器使用不同的**元编程技术**把方法和数据属性注入要构建的类
 </span>

**使用 `namedtuple` 构建 `Coordinate` 类**

- `namedtuple` 是一个工厂方法，使用指定的名称和字段构建 tuple 子类

In [4]:
from collections import namedtuple
Coordinate = namedtuple('Coordinate', 'lat lon')

print(issubclass(Coordinate, tuple))

moscow = Coordinate(55.751244, 37.618423)
print(moscow) # *1

print(moscow == Coordinate(lat=55.751244, lon=37.618423)) # *2


True
Coordinate(lat=55.751244, lon=37.618423)
True


*1 有用的 `__repr__`(输出可读的字符串)

*2 有意义的 `__eq__`

---

**`typing.NamedTuple` 还可以为各个字段添加类型注解**

In [6]:
import typing

# <= 3.5
Coordinate = typing.NamedTuple(
    'Coordinate', [('lat', float), ('lon', float)]
)

print(issubclass(Coordinate, tuple))

typing.get_type_hints(Coordinate)

True


{'lat': float, 'lon': float}

In [7]:
# >= 3.6 可以使用下面的方式

Coordinate = typing.NamedTuple(
    'Coordinate', lat=float, lon=float
)


print(typing.get_type_hints(Coordinate))

{'lat': <class 'float'>, 'lon': <class 'float'>}


后面这种方式可读性高，而且可以通过映射指定字段以及其类型，再使用 `**fields_and_types`拆包


----

3.6 开始，`typing.NamedTuple` 也可以在 `class` 语句中使用，可读性高，而且方便覆盖方法和添加新方法

In [ ]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self) -> str:
        ns = 'N' if self.lat >= 0 else 'S'
        ew = 'E' if self.lon >= 0 else 'W'
        return f"{abs(self.lat):.1f}°{ns} {abs(self.lon):.1f}°{ew}"
    
print(issubclass(Coordinate, NamedTuple))

上面的这段代码，虽然 `NamedTuple` 出现在超类的位置上，<mark>但其实它并不是超类</mark>

<span style="color:green"> `typing.NamedTuple`使用<mark>元类</mark>这一高级功能创建用户类</span>，可以通过下面的代码对其验证

In [16]:
issubclass(Coordinate, NamedTuple)

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union

In [17]:
issubclass(Coordinate, tuple) # True

True

与 `typing.NamedTuple` 一样，dataclass装饰器也支持声明实例属性，dataclass 装饰器读取变量注解，自动为构建的类生成方法


In [18]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        ew = 'E' if self.lon >= 0 else 'W'
        return f"{abs(self.lat):.1f}°{ns} {abs(self.lon):.1f}°{ew}"


----

可以看到，`@dataclass` 装饰器不依赖于继承或者元类

---

**3个数据类构建器的部分功能**

<img src="./assets/ch05/3-constructor-func.png">

可以看到，后面两列构建的类有一个`__annotations__`属性，存放字段的类型提示，<mark>然而，不建议直接读取 `__annotations__`属性。</mark>

<span style="color:green"> 推荐使用 `inspect.get_annotations(MyClass)` (>=Python3.10) 或者 typing.get_type_hints(MyClass)（3.5 - 3.9）获取类型信息 </span>

# TODO